MONTE CARLO METHOD USING PARALLEL PROCESSING

In [1]:
import ipyparallel as ipp
c = ipp.Client()
c.ids

[0, 1, 2, 3]

In [2]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
print("Hello! I'm rank %d from %d running in total..." % (comm.rank, comm.size))

[stdout:0] Hello! I'm rank 0 from 4 running in total...


[stdout:2] Hello! I'm rank 2 from 4 running in total...


[stdout:1] Hello! I'm rank 1 from 4 running in total...


[stdout:3] Hello! I'm rank 3 from 4 running in total...


In [8]:
%%px
import numpy as np
from mpi4py import MPI

# MPI initialization
comm = MPI.COMM_WORLD
size = comm.Get_size() # number of processes
rank = comm.Get_rank() # rank of current process

n = 1000000
n_local = n // size

# if n is not divisible by size, one of the processes needs to calculate more elements  
if rank == 0:
    n_local = n - n_local * (size - 1)

# monte carlo integration
x = np.random.rand(n_local)
y = np.random.rand(n_local)
z = np.sqrt(x**2 + y**2)

n_success = np.sum(z < 1) 

print("Rank %d: n_success = %d" % (rank, n_success))
print("n_local = %d" % n_local)

# gather results
reduced_n_success = np.array(0)
comm.Reduce(n_success, reduced_n_success, op=MPI.SUM, root=0) # send n_success of every process to process 0 and sum it there


print("Rank %d: pi_estimate = %f" % (rank, reduced_n_success / n * 4)) # will only be != 0 for process 0


[stdout:2] Rank 2: n_success = 196390
n_local = 250000
Rank 2: pi_estimate = 0.000000


[stdout:3] Rank 3: n_success = 196282
n_local = 250000
Rank 3: pi_estimate = 0.000000


[stdout:0] Rank 0: n_success = 196696
n_local = 250000
Rank 0: pi_estimate = 3.142316


[stdout:1] Rank 1: n_success = 196211
n_local = 250000
Rank 1: pi_estimate = 0.000000
